#### (1) Gradient Operators - Roberts, Prewitt, Sobel, Scharr

In [2]:
import cv2
import numpy as np

def Roberts2(grey):
    mask_p = np.array([[-1,0,0],[0,1,0],[0,0,0]])
    mask_m = np.array([[0,0,-1],[0,1,0],[0,0,0]])
    dp = cv2.filter2D(grey, -1, mask_p)
    dm = cv2.filter2D(grey, -1, mask_m)
    grad = np.sqrt(dp**2 + dm**2)
    grad = cv2.normalize(grad, None, 0, 1, cv2.NORM_MINMAX)
    return grad


def Prewitt2(grey):
    mask_y = np.array([[-1,-1,-1],[0,0,0],[1,1,1]])
    mask_x = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
    dx = cv2.filter2D(grey, -1, mask_x)
    dy = cv2.filter2D(grey, -1, mask_y)
    grad = np.sqrt(dx**2 + dy**2)
    grad = cv2.normalize(grad, None, 0, 1, cv2.NORM_MINMAX)
    return grad


def Sobel2(grey):
    mask_y = np.array([[-1,-2,-1],[0,0,0],[1,2,1]])
    mask_x = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
    dx = cv2.filter2D(grey, -1, mask_x)
    dy = cv2.filter2D(grey, -1, mask_y)
    # dx = cv2.Sobel(grey, cv2.CV_32F, 1, 0, ksize=3)
    # dy = cv2.Sobel(grey, cv2.CV_32F, 0, 1, ksize=3)
    grad = np.sqrt(dx**2 + dy**2)
    grad = cv2.normalize(grad, None, 0, 1, cv2.NORM_MINMAX)
    return grad


def Scharr2(grey):
    mask_y = np.array([[-3,-10,-3],[0,0,0],[3,10,3]])
    mask_x = np.array([[-3,0,3],[-10,0,10],[-3,0,3]])
    dx = cv2.filter2D(grey, -1, mask_x)
    dy = cv2.filter2D(grey, -1, mask_y)
    # dx = cv2.Scharr(grey, cv2.CV_32F, 1, 0)
    # dy = cv2.Scharr(grey, cv2.CV_32F, 0, 1)
    grad = np.sqrt(dx**2 + dy**2)
    grad = cv2.normalize(grad, None, 0, 1, cv2.NORM_MINMAX)
    return grad


image = cv2.imread('../data/Lena.png', 0).astype(np.float32)/255

grad_roberts = Roberts2(image)
grad_prewitt = Prewitt2(image)
grad_sobel = Sobel2(image)
grad_scharr = Scharr2(image)


cv2.imshow('Roberts', grad_roberts)
cv2.imshow('Prewitt', grad_prewitt)
cv2.imshow('Sobel', grad_sobel)
cv2.imshow('Scharr', grad_scharr)
cv2.waitKey()
cv2.waitKey()
cv2.destroyAllWindows()

#### (2) Laplacian Masks

In [3]:
def Laplacian2(grey, mask):
    d = cv2.filter2D(grey, -1, mask)
    d = np.abs(d)
    d = cv2.normalize(d, None, 0, 1, cv2.NORM_MINMAX)
    return d


image = cv2.imread('../data/Lena.png', 0).astype(np.float32)/255

mask_1 = np.array([[0,-1,0],[-1,4,-1],[0,-1,0]])
mask_2 = np.array([[-1,-1,-1],[-1,8,-1],[-1,-1,-1]])

lap1 = Laplacian2(image, mask_1)
lap2 = Laplacian2(image, mask_2)

cv2.imshow('Laplacian1', lap1)
cv2.imshow('Laplacian2', lap2)
cv2.waitKey()
cv2.destroyAllWindows()

#### (5) Laplacian of Gaussian

In [27]:
import cv2
import numpy as np

# 1단계
image = cv2.imread('../data/lena.png',0).astype(np.float32)/255

blur = cv2.GaussianBlur(image, (7,7), 0)

lap = cv2.subtract(image, blur)
lap = cv2.normalize(lap, None, 0, 1, cv2.NORM_MINMAX)
lap2 = cv2.Laplacian(blur, cv2.CV_32F, None, ksize=3)
lap2 = cv2.normalize(lap2, None, 0, 1, cv2.NORM_MINMAX)

cv2.imshow('DoG', lap)
cv2.imshow('LoG', lap2)
cv2.waitKey()
cv2.destroyAllWindows()

> 전처리 필터는 다른 그레디언트 오퍼레이터에도 유효하다.

`cv2.Sobel(src, ddepth, dx, dy[, dst[, ksize[, scale[, delta[, borderType]]]]]) → dst`
Parameters:	
>- src – input image
>- ddepth – output image의 depth, -1이면 input image와 동일.
>- dx – x축 미분 차수.
>- dy – y축 미분 차수.
>- ksize – kernel size(ksize x ksize)

만약 ksize가 -1이면 3x3 Scharr filter가 적용된다.

In [24]:
image = cv2.imread('../data/lena.png',0).astype(np.float32)/255

blur = cv2.GaussianBlur(image, (7,7), 0)

dx = cv2.Sobel(blur, -1, 1, 0, ksize=3)
dy = cv2.Sobel(blur, -1, 0, 1, ksize=3)
#dx = cv2.Scharr(blur, -1, 1, 0)
#dy = cv2.Scharr(blur, -1, 0, 1)
grad = np.sqrt(dx**2 + dy**2)
grad = cv2.normalize(grad, None, 0, 1, cv2.NORM_MINMAX)
grad = (grad*255).astype(np.uint8)

_, thrs = cv2.threshold(grad,0,255,cv2.THRESH_BINARY|cv2.THRESH_OTSU)

cv2.imshow('edges', thrs)
cv2.waitKey()
cv2.waitKey()
cv2.destroyAllWindows()

#### (3) Canny

`cv2.Canny(image, threshold1, threshold2[, edges[, apertureSize[, L2gradient]]]) → edges`

Parameters:	
>- image – 8-bit input image
>- threshold1 – Hysteresis Thredsholding 작업에서의 min 값
>- threshold2 – Hysteresis Thredsholding 작업에서의 max 값


In [29]:
image = cv2.imread('../data/Lena.png', 0)

edges = cv2.Canny(image, 50, 100)  # CV_8U 입력만 유효

cv2.imshow('Canny', edges)
cv2.waitKey()
cv2.waitKey()
cv2.destroyAllWindows()

In [30]:
image = cv2.imread('../data/sudoku.png', 0)

edges = cv2.Canny(image, 50, 100)  # CV_8U 입력만 유효

cv2.imshow('Canny', edges)
cv2.waitKey()
cv2.waitKey()
cv2.destroyAllWindows()